In [1]:
import librosa
import numpy as np
import os
import math
import operator
from sklearn.cluster import KMeans
import hmmlearn.hmm
import functionMFCC as func

C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\PhuongDong\miniconda3\envs\voice\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [13]:
class_names = ["nguoi", "viet_nam", "nha", "cua", "khong", "test_nguoi", "test_viet_nam", "test_nha", "test_cua", "test_khong"]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = func.get_class_data(os.path.join("data_hmm", cname))
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
# print("vectors", all_vectors.shape)
# # Run K-Means algorithm to get clusters
# kmeans = func.clustering(all_vectors)
# print("centers", kmeans.cluster_centers_.shape)
# for cname in class_names:
#     dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

Load nguoi dataset
Load viet_nam dataset
Load nha dataset
Load cua dataset
Load khong dataset
Load test_nguoi dataset
Load test_viet_nam dataset
Load test_nha dataset
Load test_cua dataset
Load test_khong dataset


In [4]:
def train_model(train_set, n_component, startprob):
    model = hmmlearn.hmm.GMMHMM(n_components=n_component, n_mix=2, n_iter=1000, params='mtcw', init_params='mt', verbose=True)
    model.startprob_ = startprob
    X = np.concatenate(train_set)
    lengths = list([len(x) for x in train_set])
    # fit dat 
    print(X.shape, lengths, len(lengths))
    model.fit(X)
    return model

In [11]:
startprob6 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0])
startprob9 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
startprob18 = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# transmat6 = np.array([[0.05 for i in range(6)] for j in range(6)])
# transmat12 = np.array([[0.05 for i in range(12)] for j in range(12)])
# for x in range(5):
#     transmat6[x][x] = 0.4
#     transmat6[x][x+1] = 0.4
# transmat6[5][5] = 0.75
# for x in range(11):
#     transmat12[x][x] = 0.25
#     transmat12[x][x+1] = 0.25
# transmat12[11][11] = 0.45
# print(transmat6)
models = {}
models["khong"] = train_model(dataset["khong"], 9, startprob9)
models["nha"] = train_model(dataset["nha"], 6, startprob6)
models["cua"] = train_model(dataset["cua"], 6, startprob6)
models["nguoi"] = train_model(dataset["nguoi"], 9, startprob9)
models["viet_nam"] = train_model(dataset["viet_nam"], 18, startprob18)

(1887, 36) [25, 21, 24, 27, 25, 21, 18, 24, 24, 21, 24, 29, 15, 31, 19, 21, 20, 25, 29, 32, 31, 23, 18, 35, 23, 21, 24, 28, 33, 28, 34, 23, 17, 17, 17, 23, 25, 25, 22, 25, 28, 22, 24, 19, 17, 21, 19, 23, 19, 35, 26, 24, 27, 21, 20, 23, 20, 21, 24, 16, 25, 19, 26, 21, 22, 26, 27, 19, 17, 35, 28, 22, 25, 29, 31, 21, 19, 21, 13, 25] 80


         1     -194321.2065             +nan
         2     -181868.8159      +12452.3906
         3     -179934.8121       +1934.0038
         4     -179441.3786        +493.4335
         5     -179221.2356        +220.1430
         6     -179050.1602        +171.0754
         7     -178943.4917        +106.6685
         8     -178861.1446         +82.3471
         9     -178814.8462         +46.2984
        10     -178795.6645         +19.1817
        11     -178789.2396          +6.4249
        12     -178786.7158          +2.5238
        13     -178784.3242          +2.3916
        14     -178782.4220          +1.9022
        15     -178779.4063          +3.0157
        16     -178759.5814         +19.8249
        17     -178752.9278          +6.6536
        18     -178751.1396          +1.7882
        19     -178750.2828          +0.8568
        20     -178749.4028          +0.8800
        21     -178748.3164          +1.0864
        22     -178746.9065          +1.4099
        23

(2541, 36) [21, 22, 30, 23, 17, 23, 28, 47, 32, 24, 42, 32, 33, 36, 25, 20, 25, 45, 22, 31, 28, 17, 42, 27, 19, 20, 33, 16, 36, 31, 39, 17, 33, 31, 28, 35, 32, 38, 18, 43, 29, 25, 22, 25, 21, 24, 32, 23, 62, 33, 22, 18, 33, 22, 16, 18, 54, 46, 52, 45, 22, 40, 32, 25, 26, 41, 32, 43, 43, 43, 26, 59, 40, 45, 32, 50, 47, 59, 32, 21] 80


         1     -247931.3476             +nan
         2     -234175.3750      +13755.9726
         3     -231324.9797       +2850.3953
         4     -230657.2782        +667.7014
         5     -230424.7528        +232.5254
         6     -230265.8175        +158.9353
         7     -230153.4452        +112.3722
         8     -230090.3137         +63.1315
         9     -230025.2847         +65.0290
        10     -229942.4606         +82.8241
        11     -229902.2405         +40.2200
        12     -229886.6489         +15.5916
        13     -229875.2881         +11.3608
        14     -229868.2037          +7.0844
        15     -229856.1385         +12.0652
        16     -229834.4069         +21.7316
        17     -229825.4336          +8.9733
        18     -229817.6873          +7.7463
        19     -229813.1747          +4.5127
        20     -229808.5248          +4.6499
        21     -229806.1780          +2.3468
        22     -229804.6859          +1.4920
        23

(1872, 36) [55, 18, 24, 23, 17, 20, 22, 32, 25, 21, 20, 62, 33, 15, 18, 15, 17, 16, 18, 16, 18, 17, 62, 20, 23, 23, 48, 19, 22, 20, 29, 18, 55, 90, 62, 22, 17, 31, 18, 16, 39, 29, 26, 18, 76, 17, 12, 12, 26, 23, 23, 23, 18, 14, 16, 19, 20, 24, 16, 17, 12, 12, 26, 23, 23, 14, 24, 16, 20, 24, 16, 23, 18, 18, 18] 75


         1     -204039.8980             +nan
         2     -194490.2631       +9549.6349
         3     -193127.3154       +1362.9477
         4     -192705.0756        +422.2397
         5     -192476.1334        +228.9422
         6     -192271.5183        +204.6151
         7     -192040.9683        +230.5501
         8     -191793.4662        +247.5021
         9     -191726.0408         +67.4254
        10     -191648.3157         +77.7252
        11     -191575.0043         +73.3113
        12     -191461.3215        +113.6828
        13     -191436.3551         +24.9664
        14     -191425.3773         +10.9778
        15     -191412.9311         +12.4462
        16     -191398.2864         +14.6447
        17     -191359.7463         +38.5401
        18     -191346.6549         +13.0914
        19     -191333.2550         +13.4000
        20     -191320.0608         +13.1942
        21     -191310.7606          +9.3002
        22     -191305.4330          +5.3276
        23

(2020, 36) [29, 30, 15, 43, 30, 30, 14, 25, 24, 30, 35, 23, 17, 24, 21, 31, 22, 28, 29, 28, 19, 16, 16, 31, 24, 22, 19, 13, 21, 23, 30, 19, 32, 47, 18, 34, 26, 21, 10, 23, 29, 25, 21, 25, 25, 12, 27, 24, 37, 22, 26, 29, 17, 26, 24, 33, 36, 22, 21, 30, 31, 31, 22, 22, 26, 23, 36, 32, 23, 17, 32, 28, 21, 32, 21, 16, 25, 26, 35, 18] 80


         1     -212912.7349             +nan
         2     -201390.0720      +11522.6629
         3     -198472.7454       +2917.3266
         4     -197771.5164        +701.2290
         5     -197448.8344        +322.6821
         6     -197325.2673        +123.5670
         7     -197267.5257         +57.7416
         8     -197217.0289         +50.4968
         9     -197179.5979         +37.4310
        10     -197163.0935         +16.5044
        11     -197152.1570         +10.9365
        12     -197146.1923          +5.9648
        13     -197143.2232          +2.9690
        14     -197140.6841          +2.5391
        15     -197138.3872          +2.2968
        16     -197136.2541          +2.1332
        17     -197134.1281          +2.1260
        18     -197131.6955          +2.4326
        19     -197129.5576          +2.1379
        20     -197127.0408          +2.5168
        21     -197124.2143          +2.8265
        22     -197120.9768          +3.2376
        23

(3536, 36) [52, 43, 35, 55, 34, 55, 47, 51, 41, 49, 34, 58, 41, 41, 75, 30, 50, 53, 39, 42, 40, 42, 49, 40, 39, 36, 40, 41, 54, 47, 46, 55, 38, 49, 37, 35, 45, 39, 58, 42, 40, 43, 34, 31, 47, 48, 37, 58, 38, 37, 50, 41, 37, 43, 50, 49, 48, 44, 51, 32, 42, 51, 42, 37, 30, 50, 45, 35, 39, 55, 47, 45, 39, 40, 61, 56, 42, 40, 46, 39] 80


         1     -368365.2592             +nan
         2     -340005.6032      +28359.6559
         3     -335794.8904       +4210.7129
         4     -334705.6597       +1089.2307
         5     -334041.4165        +664.2432
         6     -333636.9211        +404.4953
         7     -333246.0223        +390.8988
         8     -332969.0589        +276.9634
         9     -332753.7561        +215.3028
        10     -332536.2783        +217.4779
        11     -332302.7407        +233.5376
        12     -332125.9806        +176.7601
        13     -331869.8248        +256.1558
        14     -331662.6982        +207.1266
        15     -331496.9305        +165.7677
        16     -331372.1606        +124.7699
        17     -331215.0598        +157.1008
        18     -331021.5240        +193.5357
        19     -330879.2753        +142.2487
        20     -330734.2353        +145.0400
        21     -330595.4663        +138.7690
        22     -330518.8361         +76.6301
        23

In [22]:
print("Testing")
for true_cname in class_names:
    if true_cname[:4] == "test":
        result = 0
        length = 0
        for O in dataset[true_cname]:
            length+=1
            score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
            if true_cname[5:] == max(score.items(), key=operator.itemgetter(1))[0]:
                result = result + 1
            else: 
                print(true_cname, score, length, max(score.items(), key=operator.itemgetter(1))[0])
        print('Acc ' + true_cname[5:] , result/length, result, length)

Testing
Acc nguoi 1.0 20 20
Acc viet_nam 1.0 20 20
Acc nha 1.0 20 20
Acc cua 1.0 20 20
Acc khong 1.0 20 20


In [7]:
import pickle

In [8]:
with open("modelHMM.pkl", "wb") as file:
    pickle.dump(models, file)